In [1]:
import os
import pathlib
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, LLM, Process
from crewai_tools import RagTool
from langchain_community.tools import tool

In [2]:
@tool("PdfRAGTool")
def pdf_information_extractor_tool():
  """ Useful for retrieving and analyzing data from a PDF """
  return eval("Extracted data from PDF")

@tool("FastAPIDocsRAGTool")
def fastapi_documentation_extractor_tool():
  """ Useful for retrieving and analyzing data from FastAPI documentation """
  return eval("Extracted data from FastAPI documentation")

@tool("ScipyDocsRAGTool")
def scipy_documentation_extractor_tool():
  """ Useful for retrieving and analyzing data from Scipy documentation """
  return eval("Extracted data from Scipy documentation")

In [3]:
project_base_directory = pathlib.Path().resolve()
env_file_location = os.path.join(project_base_directory, ".env")
print(f"project_base_directory: {env_file_location}")

load_dotenv(dotenv_path=env_file_location)

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
  os.environ["OPENAI_API_KEY"] = openai_api_key
  print(f"OPENAI_API_KEY loaded: {openai_api_key}")
else:
  print("Error: OPENAI_API_KEY not found in .env file")

llm_instance = LLM(model="gpt-4o-mini")

project_base_directory: /home/jsamyce/TopicosIA/ai-topics-2-2024/6.multiagent/work-23-10-2024/.env
OPENAI_API_KEY loaded: sk-O00jej-yoOS6QEaoFVFRqaVY1nWAGentEJBZBzOyhMT3BlbkFJwFjQiZ8BmEs1knqfD3W8xVZcYb5ZDqVpVzNdJpotAA


In [4]:
document_analyzer_agent = Agent(
  role="DocumentAnalyzerAgent",
  goal="Analyze documents and extract relevant information for coding implementation.",
  backstory="You are an expert in analyzing technical papers and documentation.",
  tools=[pdf_information_extractor_tool, fastapi_documentation_extractor_tool, scipy_documentation_extractor_tool],
  llm=llm_instance,
)

In [5]:
technical_paper_rag_tool = RagTool(source_type="pdf", file_path=os.path.join(project_base_directory, "technical_document.pdf"))
fastapi_documentation_rag_tool = RagTool(source_type="web", url="https://fastapi.tiangolo.com/")
scipy_documentation_rag_tool = RagTool(source_type="web", url="https://docs.scipy.org/doc/")

In [6]:
analyze_technical_paper_task = Task(
  description="Read and summarize key information from the technical paper for coding implementation.",
  expected_output="A concise summary with the technical details necessary for implementation.",
  agent=document_analyzer_agent,
  tools=[technical_paper_rag_tool],
  output_file="technical_paper_summary.txt"
)

extract_fastapi_docs_task = Task(
  description="Extract relevant information from the FastAPI documentation.",
  expected_output="A detailed summary of FastAPI usage relevant to the implementation.",
  agent=document_analyzer_agent,
  tools=[fastapi_documentation_rag_tool],
  output_file="fastapi_documentation_summary.txt"
)

extract_scipy_docs_task = Task(
  description="Extract relevant information from the SciPy documentation.",
  expected_output="A detailed summary of SciPy usage relevant to the implementation.",
  agent=document_analyzer_agent,
  tools=[scipy_documentation_rag_tool],
  output_file="scipy_documentation_summary.txt"
)

In [7]:
python_code_generator_agent = Agent(
  role="PythonCodeGeneratorAgent",
  goal="Generate Python code using SciPy and FastAPI based on the provided technical paper information.",
  backstory="You are a senior Python developer specialized in implementing code from technical specifications using FastAPI and SciPy.",
  allow_code_execution=False,
  llm=llm_instance,
)

In [8]:
generate_python_code_task = Task(
  description="Generate Python code using the information from the paper, FastAPI, and SciPy documentation.",
  expected_output="A Python file implementing the code as described in the paper using FastAPI and SciPy.",
  agent=python_code_generator_agent,
  output_file="generated_implementation.md"
)

In [9]:
code_simulation_task = Task(
  description="Using the analysis provided, simulate code execution and provide expected results.",
  expected_output="Simulated code execution results based on analysis.",
  agent=python_code_generator_agent
)

In [10]:
task_execution_process = Process.sequential

development_crew = Crew(
  agents=[document_analyzer_agent, python_code_generator_agent],
  tasks=[analyze_technical_paper_task, extract_fastapi_docs_task, extract_scipy_docs_task, generate_python_code_task, code_simulation_task],
  process=task_execution_process,
  verbose=True
)

In [11]:
result = development_crew.kickoff()

print("############")
print(result)

# Agent: DocumentAnalyzerAgent
## Task: Read and summarize key information from the technical paper for coding implementation.


# Agent: DocumentAnalyzerAgent
## Thought: I need to analyze a technical paper and extract relevant information for coding implementation. To do this, I will use the PdfRAGTool to read and gather the necessary details from the document.
## Using tool: PdfRAGTool
## Tool Input: 
"{\"name\": \"Technical Paper\", \"description\": \"Extract key implementation details\", \"args_schema\": \"BaseModel\", \"return_direct\": true}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: DocumentAnalyzerAgent
## Thought: Thought: I need to read a technical paper to extract key information for implementation. I should proceed by using the PdfRAGTool to analyze the document.
## Using tool: PdfRAGTool
## Tool Input: 
"{\"name\": \"Technical Paper\", \"description\": \"Analyze technical paper for coding implementation details\", \"args_sch